In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import sympy as sym

# 2D Navier-Stokes equations

Note que las primeras derivadas de una función de dos variables, $x,y$ están dadas por 
$$ \frac{\partial f}{\partial x} = \frac{f(x+h,y)-f(x,y)}{h} = \frac{f_{i+1,j}-f_{i,j}}{h} = \frac{f_{i+1,j}-f_{i-1,j}}{2h} $$
$$ \frac{\partial f}{\partial y} = \frac{f(x,y+h)-f(x,y)}{h} = \frac{f_{i,j+1}-f_{i,j}}{h} = \frac{f_{i,j+1}-f_{i,j-1}}{2h} $$
$$ \frac{\partial^2 f}{\partial x^2} = \frac{f(x-h,y)-2f(x,y)+f(x+h,y)}{h^2} = \frac{f_{i-1,j}-2f_{i,j}+f_{i+1,j}}{h^2}$$
$$ \frac{\partial^2 f}{\partial y^2} = \frac{f(x,y-h)-2f(x,y)+f(x,y+h)}{h^2} = \frac{f_{i,j-1}-2f_{i,j}+f_{i,j-1}}{h^2}$$

Buscamos resolver las siguientes ecuaciones diferenciales para $u(x,y), w(x,y)$
$$-w = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial ^2 y}$$
$$\nu \left( \frac{\partial^2 w}{\partial x^2} + \frac{\partial^2 w}{\partial ^2 y} \right) =\frac{\partial u}{\partial y} \frac{\partial w}{\partial x} - \frac{\partial u}{\partial x}\frac{\partial w}{\partial y} $$
Utilizando el método de diferencias finitas esto queda igual a 

$$-w_{i,j} = \frac{u_{i-1,j}-2u_{i,j}+u_{i+1,j}}{h^2} + \frac{u_{i,j-1}-2u_{ij}+u_{i,j+1}}{h^2} = \frac{u_{i-1,j}+u_{i+1,j}+u_{i,j-1}+u_{i,j+1}-4u_{i,j}}{h^2} $$

$$\nu \left( \frac{w_{i-1,j}+w_{i+1,j}+w_{i,j-1}+w_{i,j+1}-4w_{i,j}}{h^2} \right) = \frac{u_{i,j+1}-u_{i,j-1}}{2h} \frac{w_{i+1,j}-w_{i-1,j}}{2h}  - \frac{u_{i+1,j}-u_{i-1,j}}{2h}\frac{w_{i,j+1}-w_{i,j-1}}{2h} $$


De esta primera ecuación se obtiene

$$ u_{i,j} = \frac{1}{4} \left(u_{i-1,j}+u_{i+1,j}+u_{i,j-1}+u_{i,j+1} + h^2 w_{i,j} \right) $$
De la segunda se tiene,
$$ w_{i,j} = \frac{1}{4} \left(w_{i-1,j}+w_{i+1,j}+w_{i,j-1}+w_{i,j+1} \right) - \frac{1}{16 \nu} (u_{i,j+1}-u_{i,j-1})(w_{i+1,j}-w_{i-1,j}) +  \frac{1}{16 \nu}(u_{i+1,j}-u_{i-1,j})(w_{i,j+1}-w_{i,j-1}) $$
Donde, en unidades normales se tiene que $R= \frac{1}{\nu} = \frac{V_0 h}{\nu}$

# Condiciones a la frontera

En esta ocasión, se tratará un obstáculo rectangular entre $x_1$ y $x_2$ y $y_1$ y $y_2$. Consideremos lo que pasa a lo largo de sus extremos verticales, es decir en $x=x_1, x=x_2$. La vorticidad está dada por 
$$w = \frac{\partial v_y}{\partial x} -\frac{\partial v_x}{\partial y} $$
Sin embargo, como $x$ es constante se tendrá que 
$$w = -\frac{\partial v_x}{\partial y} = - \frac{\partial^2 u}{\partial y^2}$$
Note tambien que por la serie de Taylor se tiene 
$$u(x,y+h) = u(x,y) + \frac{\partial u}{\partial y}(x,y) h + \frac{\partial^2 u}{\partial y^2}(x,y) \frac{h^2}{2}$$
Adicionalmente se tiene que la velocidad en la parte superior del obstáculo será nula, es decir

$$v_x = 0 = \frac{\partial u}{\partial y}$$
Todo esto permite simplificar la segunda derivada como

$$\frac{\partial^2 u}{\partial y^2} =  \frac{2}{h^2} \left( u(x,y+h) - u(x,y) \right) = 2 \frac{u_{i,j+1}-u_{i,j}}{h^2}$$
Haciendo un análisis similar para el otro lado del obstáculo se tiene

**Derecha:** $w_{i,j} = - 2 \frac{u_{i,j+1}-u_{i,j}}{h^2}$

**Izquierda:** $w_{i,j} = - 2 \frac{u_{i,j-1}-u_{i,j}}{h^2}$

In [ ]:
V0 = 1
nu = 0.2

In [ ]:
x1 = 0.05
x2 = 0.25
y1 = 0.05
y2 = 0.30
L = 1

In [ ]:
N = 50
h= 0.02
omega = 0.9
R = V0*h/nu

In [ ]:
u = np.zeros(N+1,N+1)
w = np.zeros(N+1,N+1)

In [ ]:
def Limits(L):
    for i in range(0,N+1):
        for j in range(0,N+1):
            w[i,j] = 0
            u[i,j] = j*V0
    for i in range(0,N+1):
        u[i,N] = u[i,N-1]+V0*h
        u[i,N-1] = 0
    for j in range(0,N+1):
        u[1,j] = u[0,j]
        w[0,j] =0
    for i in range(0,N+1):
        if i<=0 or i>= L:
            u[i,0] =0
            w[i,0] =0
    for j in range(1,N):
        w[N,j] = w[N-1,j]
        u[N,j] = u[N-1,j]
        
def Obstaculo():
    for j in range()